In [19]:
import pandas as pd
from datetime import datetime
from sklearn.neighbors import KernelDensity
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()

# url = 'https://www.wyldata.com/iframes/wsk-owsr1-ranking?gender=1&discipline=1'
urls=[
    'https://www.wyldata.com/iframes/wsk-owsr1-ranking?gender=1&discipline=1','https://www.wyldata.com/iframes/wsk-owsr1-ranking?gender=1&discipline=2',
    'https://www.wyldata.com/iframes/wsk-owsr1-ranking?gender=0&discipline=1','https://www.wyldata.com/iframes/wsk-owsr1-ranking?gender=0&discipline=2'
]
# driver.get(url)

def click_and_extract():
    try:
        flex_divs = driver.find_elements(By.CSS_SELECTOR, 'div.flex.relative.col-span-4')
        print(f"Found {len(flex_divs)} 'flex' div elements.")

        results = []

        for i, div in enumerate(flex_divs):
            try:
                if i==20:
                    break
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(div)
                ).click()
                time.sleep(2)


                table = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'table'))
                )


                rows = table.find_elements(By.TAG_NAME, 'tr')



                athlete_data = []

                for row in rows:
                    cells = row.find_elements(By.TAG_NAME, 'td')
                    cell_data = [cell.text.split('\n')[0] if '\n' in cell.text else cell.text for cell in cells]
                    athlete_data.append(cell_data)

                name = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'h2.text-2xl'))
                ).text

                athlete_name = name.split('\n')[1] if '\n' in name else name

                athlete_country = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'p.text-base'))
                ).text




                results.append({
                    'name': athlete_name,
                    'country': athlete_country,
                    'data': athlete_data
                })

                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.text-sm'))
                ).click()

                time.sleep(1)

            except Exception as e:
                print(f"Error clicking on div {i+1}: {e}")

        return results

    except Exception as e:
        print(f"An error occurred while locating 'flex' div elements: {e}")
        return []

datas= []
for i in range(4):
    url = urls[i]
    driver.get(url)
    all_athlete_data = click_and_extract()
    datas.append(all_athlete_data)

driver.quit()


Found 44 'flex' div elements.
Found 44 'flex' div elements.
Found 44 'flex' div elements.
Found 44 'flex' div elements.


In [11]:
print(datas[3])

[{'name': 'Momiji Nishiya', 'country': 'Japan', 'data': [[], ['2024-06-23', 'OQS Budapest 2024', '15', '33384'], ['2024-05-19', 'OQS Shanghai 2024', '9', '69824'], ['2024-03-10', 'World Skateboarding Tour - Street Dubai', '10', '10258'], ['2023-12-17', 'WST Street World Championship 2023 Tokyo', '3', '54400'], ['2023-09-16', 'World Skateboarding Tour - Street Lausanne 2023', '1', '50000'], ['2023-06-25', 'World Skateboarding Tour Stop Rome Street 2023', '8', '15086'], ['2023-02-05', 'Street World Championships Sharjah 2022', '3', '54400'], ['2022-07-03', 'World Street Skateboarding Rome 2022', '2', '40000'], ['Overall', '', '', '302008']]}, {'name': 'Rayssa Leal', 'country': 'Brazil', 'data': [[], ['2024-05-19', 'OQS Shanghai 2024', '1', '260000'], ['2024-03-10', 'World Skateboarding Tour - Street Dubai', '8', '15086'], ['2023-12-17', 'WST Street World Championship 2023 Tokyo', '2', '64000'], ['2023-09-16', 'World Skateboarding Tour - Street Lausanne 2023', '7', '17748'], ['2023-06-25'

In [94]:
# Function to convert results to dataframes
import pandas as pd


def results_to_dataframe(results):
    rows = []
    for result in results:
        athlete_name = result['name']
        athlete_country = result['country']
        for entry in result['data'][1:]:
            if entry:
                row = {
                    'Athlete': athlete_name,
                    'Country': athlete_country,
                    'Date': entry[0],
                    'Event': entry[1],
                    'Rank': entry[2],
                    'Points': entry[3]
                }
                if row['Date'] == 'Overall':
                    continue
                rows.append(row)

    df = pd.DataFrame(rows, columns=['Athlete', 'Country', 'Date', 'Event', 'Rank', 'Points'])
    return df


# Convert results to dataframes
all_dataframes = []
for data in datas:
    all_dataframes.append(results_to_dataframe(data))

# Print the dataframes
# for df in all_dataframes:
#     print(df)

In [25]:
all_dataframes[0].to_csv('men_park.csv', index=False)
all_dataframes[1].to_csv('men_street.csv', index=False)
all_dataframes[2].to_csv('women_park.csv', index=False)
all_dataframes[3].to_csv('women_street.csv', index=False)

In [6]:
import pandas as pd

csv_files = ['men_park.csv', 'men_street.csv', 'women_park.csv', 'women_street.csv']


all_dataframes_temp = []

for file in csv_files:
    df = pd.read_csv(file)
    all_dataframes_temp.append(df)

In [7]:
print(all_dataframes_temp[0].head())

         Athlete        Country        Date  \
0  Gavin Bottger  United States  2024-06-23   
1  Gavin Bottger  United States  2024-05-19   
2  Gavin Bottger  United States  2024-03-03   
3  Gavin Bottger  United States  2023-10-08   
4  Gavin Bottger  United States  2023-05-28   

                                              Event  Rank  Points  
0                                 OQS Budapest 2024     4  115940  
1                                 OQS Shanghai 2024     6   91382  
2             World Skateboarding Tour - Park Dubai     2   40000  
3       WST Park World Championship 2023 Rome Ostia     1   80000  
4  World Skateboarding Tour Stop San Juan Park 2023     5   24565  


In [78]:
# def filter_by_country_and_names(df):
#     filtered_dfs = []
#     unique_countries = df['Country'].unique()
#     for country in unique_countries:
#         country_df = df[df['Country'] == country]
#         unique_names = country_df['Athlete'].unique()[:3]
#         for name in unique_names:
#             name_df = df[df['Athlete'] == name]
#             filtered_dfs.append(name_df)
#     result_df = pd.concat(filtered_dfs)
#     return result_df

# all_dataframes = [filter_by_country_and_names(df) for df in all_dataframes_temp]


In [76]:
qualified_athletes = [
    "Keegan Palmer", "Gavin Bottger", "Tom Schaar", "Augusto Akio", "Kieran Woolley",
    "Luigi Cini", "Pedro Barros", "Danny Leon", "Viktor Solmunde", "Keefer Wilson",
    "Alex Sorgente", "Hampus Winberg", "Vincent Matheron", "Steven Pineiro", "Yuro Nagahara",
    "Alessandro Mazzara", "Thomas Augusto", "Alain Kortabitarte", "Andy Macdonald", "Tyler Edtmayer", "Dallas Oberholzer",
    "Ginwoo Onodera", "Sora Shirai", "Yuto Horigome", "Jagger Eaton", "Nyjah Huston",
    "Gustavo Ribeiro", "Chris Joslin", "Richard Tury", "Matias Dell Olio", "Aurelien Giraud",
    "Kelvin Hoefler", "Giovanni Vianna", "Cordano Russell", "Matt Berger", "Ryan Decenzo",
    "Joseph Garbaccio", "Felipe Gustavo", "Vincent Milou", "Jhancarlos Gonzalez", "Mauro Iglesias",
    "Shane O'Neill", "Brandon Valjalo", "Coco Yoshizawa", "Liz Akama", "Rayssa Leal",
    "Funa Nakayama", "Chloe Covell", "Chenxi Cui", "Paige Heyn", "Poe Pinson",
    "Wenhui Zeng", "Yuanling Zhu", "Roos Zwetsloot", "Pamela Rosa", "Jazmin Alvarez",
    "Natalia Munoz", "Liv Lovelace", "Keet Oldenbeuving", "Gabi Mazetto", "Vareeraya Sukasem",
    "Mariah Duran", "Haylie Powell", "Lucie Schoonheere", "Boipelo Awuah", "Kokona Hiraki",
    "Arisa Trew", "Sakura Yosozumi", "Sky Brown", "Hinano Kusaki", "Raicca Ventura",
    "Bryce Wettstein", "Ruby Trew", "Dora Varella", "Ruby Lilley", "Isadora Pacheco",
    "Minna Stess", "Naia Laso", "Heili Sirvio", "Lilly Stoephasius", "Nana Taboulet",
    "Lola Tambling", "Fay Ebert", "Emilie Alexandre", "Julia Benedetti", "Haohao Zheng",
    "Aya Asaqas"
]


# Función para filtrar por nombres de atletas clasificados
def filter_by_qualified_athletes(df, qualified_athletes):
    return df[df['Athlete'].isin(qualified_athletes)]

all_dataframes = [filter_by_qualified_athletes(df,qualified_athletes) for df in all_dataframes_temp]

In [77]:
def print_country(df):
    filtered_dfs = []
    unique_countries = df['Country'].unique()
    for country in unique_countries:
        print(df[df['Country'] == country])

print_country(all_dataframes[3])

         Athlete Country        Date  \
8    Rayssa Leal  Brazil  2024-05-19   
9    Rayssa Leal  Brazil  2024-03-10   
10   Rayssa Leal  Brazil  2023-12-17   
11   Rayssa Leal  Brazil  2023-09-16   
12   Rayssa Leal  Brazil  2023-06-25   
13   Rayssa Leal  Brazil  2023-02-05   
14   Rayssa Leal  Brazil  2022-07-03   
53   Pamela Rosa  Brazil  2024-06-23   
54   Pamela Rosa  Brazil  2024-05-19   
55   Pamela Rosa  Brazil  2023-12-17   
56   Pamela Rosa  Brazil  2023-09-16   
57   Pamela Rosa  Brazil  2023-06-25   
58   Pamela Rosa  Brazil  2023-02-05   
59   Pamela Rosa  Brazil  2022-07-03   
74  Gabi Mazetto  Brazil  2024-06-23   
75  Gabi Mazetto  Brazil  2024-05-19   
76  Gabi Mazetto  Brazil  2024-03-10   
77  Gabi Mazetto  Brazil  2023-09-16   
78  Gabi Mazetto  Brazil  2023-06-25   
79  Gabi Mazetto  Brazil  2023-02-05   
80  Gabi Mazetto  Brazil  2022-07-03   

                                              Event  Rank  Points  
8                                 OQS Shanghai 2024

In [98]:

mini = float('inf')
for x in all_dataframes:
    # x['Points'] = pd.to_numeric(x['Points'], errors='coerce')
    x = x.dropna(subset=['Points'])
    if not x['Points'].empty:
        minval = x['Points'].min()
        mini = min(minval, mini)

print(mini)


72


In [91]:
for x in all_dataframes:
  x.dropna()

In [99]:
print(all_dataframes[3])

               Athlete      Country        Date  \
8          Rayssa Leal       Brazil  2024-05-19   
9          Rayssa Leal       Brazil  2024-03-10   
10         Rayssa Leal       Brazil  2023-12-17   
11         Rayssa Leal       Brazil  2023-09-16   
12         Rayssa Leal       Brazil  2023-06-25   
..                 ...          ...         ...   
123  Keet Oldenbeuving  Netherlands  2023-12-17   
124  Keet Oldenbeuving  Netherlands  2023-09-16   
125  Keet Oldenbeuving  Netherlands  2023-06-25   
126  Keet Oldenbeuving  Netherlands  2023-02-05   
127  Keet Oldenbeuving  Netherlands  2022-07-03   

                                               Event  Rank  Points  
8                                  OQS Shanghai 2024     1  260000  
9            World Skateboarding Tour - Street Dubai     8   15086  
10          WST Street World Championship 2023 Tokyo     2   64000  
11   World Skateboarding Tour - Street Lausanne 2023     7   17748  
12    World Skateboarding Tour Stop Rome S

In [100]:
import random
from sklearn.neighbors import KernelDensity

def obtenerkdes(df2):
  df_unique = df2.drop_duplicates(subset='Athlete', keep='first')

  #lista con los atletas y kde
  kdes=[]


  for name in df_unique['Athlete']:
    points = df2.loc[df2['Athlete'] == name, 'Points']
    dataset = points.values[:, np.newaxis]


    a_1 = dataset[0][:, np.newaxis]

    dataset = np.concatenate((a_1,dataset),axis=0)
    dataset = np.concatenate((a_1,dataset),axis=0)



    kde = KernelDensity(kernel='epanechnikov', bandwidth=1000).fit(dataset)

    kdes.append([name,kde])

  return kdes


In [101]:
def global_max(kde):

    # Puntos donde queremos evaluar la densidad
    X_d = np.linspace(0, 260000,72)[:, np.newaxis]

    # Evaluar la densidad en los puntos X_d
    log_densidad = kde.score_samples(X_d)
    densidad = np.exp(log_densidad)
    return np.max(densidad)

#     return kde
def evaluate_point(x,kde):
    punto_eval = np.array([[x]])
    log_densidad_punto = kde.score_samples(punto_eval)
    return np.exp(log_densidad_punto)

def auxiliar_v_a():
    return 260000 * np.random.uniform(0,1)

def lista_densidad(kde):
    # Puntos donde queremos evaluar la densidad
    X_d = np.linspace(0, 260000,72)[:, np.newaxis]

    # Evaluar la densidad en los puntos X_d
    log_densidad = kde.score_samples(X_d)
    return np.exp(log_densidad)

# Generar muestras usando el método de aceptación-rechazo
def accept_rejection(kde):

    while True:
        # Generar X ~ U(0, 1/c)
        X = auxiliar_v_a()
        # Generar U ~ U(0, 1)
        U = np.random.uniform(0, 1)
        # Verificar condición de aceptación
        if U <= evaluate_point(X,kde)/ global_max(kde):  # g(x) = 1 para U(0, 1)
            return X


In [102]:
def elimination_round(kdes):
  results = []

  for kde in kdes:
    # print(kdes[x])
    simulated_points = accept_rejection(kde[1])
    name=kde[0]
    # simulated_points = kde[1].sample(1)
    results.append((name, simulated_points, kde))

  results.sort(key=lambda x: x[1], reverse=True)

  return results[:12]



def final_round(results):
  final_results = []
  for name,simulated_points, kde in results:
    simulated_points2 =  accept_rejection(kde[1]) #kde[1].sample(1)
    final_results.append((name, simulated_points+simulated_points2))

  final_results.sort(key=lambda x: x[1], reverse=True)

  ret =[]

  for i in final_results:
    ret.append(i[0])

  return ret[:8]


In [103]:
def simulate_tournament():

  #lista que contiene el top 8 por cada simulacion

  men_park = []
  men_street = []
  women_park =[]
  women_street = []
  simulations = [men_park,men_street,women_park,women_street]
  i=0

  for x in all_dataframes:
    simulat = []
    kdes = obtenerkdes(x)
    for _ in range(1000):
      results1 = elimination_round(kdes)
      final_results = final_round(results1)
      simulat.append(final_results)

    simulations[i] = simulat
    i+=1

  return simulations


In [104]:
from collections import Counter

def get_top_8(simulations, num_simulations=1000):

  #lista de i listas, 1 <= i <= 8, donde cada una contiene los atletas y la cantidad de veces que quedo en el lugar i en el ranking
  counters = [Counter([simulation[i] for simulation in simulations]) for i in range(len(simulations[0]))]
  oficial_ranking = []

  for i in range(8):

    if not i:
      name, count = counters[0].most_common(1)[0]
      oficial_ranking.append([name,count/num_simulations])
      continue

    for j in range(i):
      name, count = oficial_ranking[j]
      if counters[i].get(name):
        counters[i].pop(name)

    name, count = counters[i].most_common(1)[0]

    suma = 0
    #calculamos la probabilidad de que el atleta quede en la posicion i luego de haber obtenido las posiciones anteriores
    for j in range(8-i):
      for x in counters[j+i]:
        if x == name:
          suma += counters[j+i][x]
          break

    prob = count/suma

    oficial_ranking.append([name,prob])

  suma = 0
  #prioridades que se le otorgan a cada posicion
  prob = [0.5, 0.3, 0.1 , 0.05 , 0.0125 , 0.0125 , 0.0125 , 0.0125]
  for i in range(8):
    tmp = oficial_ranking[i][1] * prob[i]
    suma+=tmp

  return oficial_ranking, suma


In [105]:
simulations = simulate_tournament()

In [106]:
p=["Skate Men Park:","Skate Men Street:","Skate Women Park:","Skate Women Street:"]
a=0

for x in simulations:
  print(p[a])
  a+=1
  top_8, precision = get_top_8(x)
  print(top_8)
  print(precision)
  print('\n')

# top_8, precision = get_top_8(simulations)

Skate Men Park:
[['Keegan Palmer', 0.45], ['Tom Schaar', 0.3949416342412451], ['Augusto Akio', 0.3274760383386581], ['Gavin Bottger', 0.36199095022624433], ['Luigi Cini', 0.39748201438848924], ['Danny Leon', 0.3291139240506329], ['Alex Sorgente', 0.5106382978723404], ['Keefer Wilson', 1.0]]
0.4222950695714449


Skate Men Street:
[['Ginwoo Onodera', 0.33], ['Sora Shirai', 0.18926553672316385], ['Aurelien Giraud', 0.26126126126126126], ['Kelvin Hoefler', 0.31955922865013775], ['Gustavo Ribeiro', 0.2863247863247863], ['Cordano Russell', 0.40816326530612246], ['Jagger Eaton', 0.45112781954887216], ['Richard Tury', 1.0]]
0.2907039469653294


Skate Women Park:
[['Sky Brown', 0.553], ['Kokona Hiraki', 0.462787550744249], ['Hinano Kusaki', 0.30078125], ['Raicca Ventura', 0.358356940509915], ['Bryce Wettstein', 0.21731748726655348], ['Ruby Lilley', 0.4], ['Minna Stess', 0.5261538461538462], ['Ruby Trew', 1.0]]
0.4901256289165255


Skate Women Street:
[['Coco Yoshizawa', 0.294], ['Chloe Covell',

In [107]:
import pycountry

# Función para obtener el código COI de un país
def get_coi_code(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

# Lista de títulos
p = ["Skate Men Park:", "Skate Men Street:", "Skate Women Park:", "Skate Women Street:"]
a = 0

# Función para obtener el país de un participante desde los dataframes
def get_country(participant_name):
    for df in all_dataframes:
        if participant_name in df['Athlete'].values:
            country = df[df['Athlete'] == participant_name]['Country'].values[0]
            return country
    return None

# Ejecutar las simulaciones e imprimir los resultados
for x in simulations:
    print(p[a])
    a += 1
    top_8, precision = get_top_8(x)

    # Agregar el código COI correspondiente a cada tupla en top_8
    for participant in top_8:
        country = get_country(participant[0])
        if country:
            coi_code = get_coi_code(country)
            if coi_code:
                participant.append(coi_code)

    print(top_8)
    print(precision)
    print('\n')


Skate Men Park:
[['Keegan Palmer', 0.45, 'AUS'], ['Tom Schaar', 0.3949416342412451, 'USA'], ['Augusto Akio', 0.3274760383386581, 'BRA'], ['Gavin Bottger', 0.36199095022624433, 'USA'], ['Luigi Cini', 0.39748201438848924, 'BRA'], ['Danny Leon', 0.3291139240506329, 'ESP'], ['Alex Sorgente', 0.5106382978723404, 'ITA'], ['Keefer Wilson', 1.0, 'AUS']]
0.4222950695714449


Skate Men Street:
[['Ginwoo Onodera', 0.33, 'JPN'], ['Sora Shirai', 0.18926553672316385, 'JPN'], ['Aurelien Giraud', 0.26126126126126126, 'FRA'], ['Kelvin Hoefler', 0.31955922865013775, 'BRA'], ['Gustavo Ribeiro', 0.2863247863247863, 'PRT'], ['Cordano Russell', 0.40816326530612246, 'CAN'], ['Jagger Eaton', 0.45112781954887216, 'USA'], ['Richard Tury', 1.0, 'SVK']]
0.2907039469653294


Skate Women Park:
[['Sky Brown', 0.553, 'GBR'], ['Kokona Hiraki', 0.462787550744249, 'JPN'], ['Hinano Kusaki', 0.30078125, 'JPN'], ['Raicca Ventura', 0.358356940509915, 'BRA'], ['Bryce Wettstein', 0.21731748726655348, 'USA'], ['Ruby Lilley', 0